In [1]:
!pip install keras tensorflow

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
from keras import backend

/Users/zkan/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/zkan/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [8]:
from imutils import paths

In [9]:
image_paths = list(paths.list_images('datasets/animals/'))

In [14]:
%matplotlib inline
import cv2
import matplotlib.pyplot as plt
import random
import numpy as np

In [15]:
random.shuffle(image_paths)

In [16]:
data = []
labels = []

for image_path in image_paths:
    image = cv2.imread(image_path)
    image = cv2.resize(image, (32, 32))
    label = image_path.split('/')[-2]
    data.append(image)
    labels.append(label)

In [17]:
data = np.array(data) / 255.0
labels = np.array(labels)

In [18]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [19]:
data.shape

(3000, 32, 32, 3)

In [20]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.core import Dense, Flatten

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
X_train, X_test, y_train, y_test = train_test_split(
    data, labels, test_size=0.2, random_state=33
)

In [23]:
model = Sequential()
model.add(
    Conv2D(
        32, 
        (3, 3), 
        input_shape=(32, 32, 3), 
        activation='relu',
        padding='same'
    )
)
model.add(Flatten())
model.add(Dense(3, activation='softmax'))

In [24]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
flatten_1 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 98307     
Total params: 99,203
Trainable params: 99,203
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [26]:
model.fit(X_train, y_train, epochs=10, validation_split=0.2)

Train on 1920 samples, validate on 480 samples
Epoch 1/10
1920/1920 [==============================] - 1s 562us/step - loss: 0.8801 - acc: 0.5495 - val_loss: 0.6872 - val_acc: 0.6646
Epoch 2/10
1920/1920 [==============================] - 1s 369us/step - loss: 0.6538 - acc: 0.7000 - val_loss: 0.6485 - val_acc: 0.6771
Epoch 3/10
1920/1920 [==============================] - 1s 388us/step - loss: 0.5181 - acc: 0.7833 - val_loss: 0.6485 - val_acc: 0.6771
Epoch 4/10
1920/1920 [==============================] - 1s 383us/step - loss: 0.4338 - acc: 0.8292 - val_loss: 0.6267 - val_acc: 0.7188
Epoch 5/10
1920/1920 [==============================] - 1s 373us/step - loss: 0.3581 - acc: 0.8917 - val_loss: 0.6371 - val_acc: 0.7104
Epoch 6/10
1920/1920 [==============================] - 1s 401us/step - loss: 0.2875 - acc: 0.9167 - val_loss: 0.6819 - val_acc: 0.6958
Epoch 7/10
1920/1920 [==============================] - 1s 361us/step - loss: 0.2477 - acc: 0.9333 - val_loss: 0.6542 - val_acc: 0.7146
E

In [27]:
y_pred = model.predict(X_test)

In [29]:
y_pred

array([[9.8355144e-01, 1.5985182e-02, 4.6332864e-04],
       [2.3220515e-02, 1.8008670e-01, 7.9669279e-01],
       [9.3757266e-01, 1.4672824e-02, 4.7754519e-02],
       ...,
       [8.0611086e-01, 1.9386464e-01, 2.4460480e-05],
       [7.2196126e-02, 5.3195006e-01, 3.9585376e-01],
       [8.7351072e-01, 1.2647997e-01, 9.3275994e-06]], dtype=float32)

In [30]:
y_pred[0]

array([9.8355144e-01, 1.5985182e-02, 4.6332864e-04], dtype=float32)

In [28]:
from sklearn.metrics import classification_report

In [36]:
print(classification_report(
    y_test.argmax(axis=1), 
    y_pred.argmax(axis=1), 
    target_names=lb.classes_))

             precision    recall  f1-score   support

       cats       0.56      0.62      0.59       200
       dogs       0.55      0.55      0.55       197
      panda       0.84      0.75      0.79       203

avg / total       0.65      0.64      0.64       600



In [38]:
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Dropout

In [41]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(3, activation='softmax'))

In [42]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 128)       36992     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 128)       147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 128)       0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 10, 10, 128)       0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 12800)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 38403     
Total para

In [44]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [45]:
model.fit(X_train, y_train, epochs=10, validation_split=0.2)

Train on 1920 samples, validate on 480 samples
Epoch 1/10
1920/1920 [==============================] - 26s 13ms/step - loss: 0.8942 - acc: 0.5214 - val_loss: 0.7916 - val_acc: 0.6021
Epoch 2/10
1920/1920 [==============================] - 28s 15ms/step - loss: 0.7451 - acc: 0.6349 - val_loss: 0.7106 - val_acc: 0.6583
Epoch 3/10
1920/1920 [==============================] - 28s 14ms/step - loss: 0.6585 - acc: 0.6849 - val_loss: 0.6454 - val_acc: 0.6937
Epoch 4/10
1920/1920 [==============================] - 27s 14ms/step - loss: 0.6018 - acc: 0.7240 - val_loss: 0.6754 - val_acc: 0.6813
Epoch 5/10
1920/1920 [==============================] - 27s 14ms/step - loss: 0.5292 - acc: 0.7578 - val_loss: 0.6974 - val_acc: 0.6917
Epoch 6/10
1920/1920 [==============================] - 26s 14ms/step - loss: 0.4881 - acc: 0.7776 - val_loss: 0.6396 - val_acc: 0.6875
Epoch 7/10
1920/1920 [==============================] - 27s 14ms/step - loss: 0.4204 - acc: 0.8141 - val_loss: 0.6444 - val_acc: 0.7083
E

In [46]:
y_pred = model.predict(X_test)

In [47]:
print(classification_report(
    y_test.argmax(axis=1), 
    y_pred.argmax(axis=1), 
    target_names=lb.classes_))

             precision    recall  f1-score   support

       cats       0.68      0.66      0.67       200
       dogs       0.63      0.63      0.63       197
      panda       0.83      0.87      0.85       203

avg / total       0.72      0.72      0.72       600



In [48]:
model.save('mymodel.h5')

In [51]:
import pickle

In [52]:
f = open('lb.pkl', 'wb')
pickle.dump(lb, f)
f.close()

In [ ]:
https://bit.ly/2OKCTYm